In [71]:
import csv
import sys
import argparse
from collections import defaultdict
import pandas as pd
import numpy as np

In [112]:
# MD HELPERS:
# vip-vip-v4
# vip-vip-v3
# vip-morphed-v3
# vip-vip-v1
# vip-vizoo-v1
# vip-vizoo-v1-pig
# vip-vizoo-v3-monkey
# vip-vizoo-v4-neurorat
# vip-head-bundle

In [122]:
EXCEL_PP_MAPPING = {
     "vip-vip-v4": "ViP V4",
    "vip-vip-v3": "ViP V3",
    "vip-morphed-v3": "Morphed V3 (*)",
    "vip-vip-v1": "ViP V1",
    "vip-vizoo-v1": "ViZoo V1",
    "vip-vizoo-v1-pig": "ViZoo V1 – Pig",
    "vip-vizoo-v3-monkey": "ViZoo V3 – Monkey",
    "vip-vizoo-v4-neurorat": "ViZoo V4 – NeuroRat",
    "vip-head-bundle": "Head bundle", 
}

In [120]:
SQL_TEMPLATE = """BEGIN;

WITH plan_id AS (
    SELECT pp.pricing_plan_id
    FROM resource_tracker_pricing_plans pp
    WHERE pp.pricing_plan_key = '{pricing_plan_key}'
      AND pp.product_name = '{product_name}'
    LIMIT 1
),
new_licensed_item AS (
    INSERT INTO licensed_items
        (licensed_resource_type, 
         pricing_plan_id, 
         product_name, 
         created, 
         modified,
         display_name, 
         key, 
         version, 
         is_hidden_on_market
        )
    SELECT
        'VIP_MODEL',
        plan_id.pricing_plan_id,
        '{product_name}',
        now(),
        now(),
        '{display_name}',
        '{license_key}',
        '{license_version}',
        {is_hidden_in_market}
    FROM plan_id
    RETURNING licensed_item_id
),
matching_resources AS (
    SELECT 
      licensed_resource_id
    FROM licensed_resources
    WHERE CAST(licensed_resource_data->'source'->>'id' AS INTEGER) IN ({id_list})
),
insert_relationships AS (
    INSERT INTO licensed_item_to_resource
        (licensed_item_id, licensed_resource_id, created, modified)
    SELECT
        new_licensed_item.licensed_item_id,
        matching_resources.licensed_resource_id,
        now(),
        now()
    FROM new_licensed_item
    CROSS JOIN matching_resources
    RETURNING licensed_item_id, licensed_resource_id
)

SELECT * FROM insert_relationships;

COMMIT;

-------------------------------------------------------------------------------
"""

In [133]:
# vip-vip-v4 - ok
# vip-vip-v3
# vip-morphed-v3 - ok
# vip-vip-v1 - ok
# vip-vizoo-v1 - ok
# vip-vizoo-v1-pig - ok
# vip-vizoo-v3-monkey 0 ok
# vip-vizoo-v4-neurorat ok
# vip-head-bundle - ok

INPUT_PRICING_PLAN_KEY = "vip-vip-v3"
INPUT_PRODUCT = "s4l"
#############################################

csv_file_path = "VIP Model licensing Sim4Life.web - mapping_table (1).csv"
df = pd.read_csv(csv_file_path)
df_filtered = df[df["PRICING"] == EXCEL_PP_MAPPING[INPUT_PRICING_PLAN_KEY]]  # <-- CHANGE PRICING PLAN!
grouped = df_filtered.groupby(["key", "version"])
assert len(grouped) > 0, "TIP: probably wrong PRICING PLAN filter"

#############################################
sql_statements = []

for (k, v), group_df in grouped:
    print(f"Group for key={k}, version={v}:")
    # Each `group_df` contains rows that share the same key and version
    
    id_list = group_df['id'].tolist()

    assert len(id_list) == len(set(id_list))
    _licensed_item_display_name = set(group_df['display_name'].tolist())
    assert len(_licensed_item_display_name) == 1
    _licensed_item_is_hidden_on_market = set(group_df['Shown on Market'].isnull().tolist())
    assert len(_licensed_item_is_hidden_on_market) == 1

    print(f"Display name: {_licensed_item_display_name}, is hidden on market {_licensed_item_is_hidden_on_market}")
    
    print(f"All IDs ({len(group_df)}) in this group:", id_list)
    print("-----------------------------------")

    _display_name_str = _licensed_item_display_name.pop()
    _is_hidden_on_market_str = _licensed_item_is_hidden_on_market.pop()
    _id_list_str = ",".join(str(i) for i in id_list)
    
    sql_block = SQL_TEMPLATE.format(
            pricing_plan_key=INPUT_PRICING_PLAN_KEY,
            product_name=INPUT_PRODUCT,
            # pricing_plan_id=PRICING_PLAN_ID,
            display_name=_display_name_str,
            license_key=k,
            license_version=v,
            is_hidden_in_market=_is_hidden_on_market_str,
            id_list=_id_list_str
        )
    sql_statements.append(sql_block)


#############################################

output_file = "test.sql"

# Write all SQL blocks to the output file (or you could print them)
with open(output_file, 'w', encoding='utf-8') as out_f:
    for block in sql_statements:
        out_f.write(block)

Group for key=CMODEL_BILLIE, version=3.0.0:
Display name: {'Billie V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: [72]
-----------------------------------
Group for key=CMODEL_BILLIE_POSABLE, version=3.1.0:
Display name: {'Billie V3.1'}, is hidden on market {False}
All IDs (2) in this group: [155, 51]
-----------------------------------
Group for key=CMODEL_BILLIE_POSABLE (NOT_SHOWN), version=3.0.0:
Display name: {'Billie V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: [50]
-----------------------------------
Group for key=CMODEL_DIZZY, version=3.0.0:
Display name: {'Dizzy V3.0 (Not shown)'}, is hidden on market {True}
All IDs (1) in this group: [60]
-----------------------------------
Group for key=CMODEL_DIZZY_POSABLE, version=3.1.0:
Display name: {'Dizzy V3.1'}, is hidden on market {False}
All IDs (2) in this group: [154, 63]
-----------------------------------
Group for key=CMODEL_DUKE, version=3.0.0:
Display name: {'Duke V3.0 (N